In [1]:
# novel_qa.py
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import faiss
import torch

# ========== モデル設定 ==========
LLM_NAME = "cyberagent/open-calm-small"
EMBED_NAME = "sentence-transformers/all-MiniLM-L6-v2"



In [2]:
# PyTorch
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))



True
NVIDIA GeForce GTX 1060 6GB


In [3]:
# ========== モデル設定 ==========
LLM_NAME = "cyberagent/open-calm-small"
EMBED_NAME = "sentence-transformers/all-MiniLM-L6-v2"

# ========== 1. LLM & トークナイザー ==========
print("Loading LLM...")
tokenizer = AutoTokenizer.from_pretrained(LLM_NAME)
tokenizer.padding_side = "left"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    LLM_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)

Loading LLM...


/home/sh1/deepl/mynovel/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/sh1/deepl/mynovel/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
# ========== 2. 埋め込みモデル ==========
print("Loading embedder...")
embedder = SentenceTransformer(EMBED_NAME)

# ========== 3. テキスト分割 ==========
with open("wagahaiwa_nekodearu.txt", encoding="utf-8") as f:
    text = f.read()

# 適度に分割（1チャンク = 約300文字）
chunk_size = 300
chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
print(f"Total chunks: {len(chunks)}")

# ========== 4. ベクトル化 & FAISS ==========
embeddings = embedder.encode(chunks, show_progress_bar=True)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)
print(f"FAISS index size: {index.ntotal}")


Loading embedder...
Total chunks: 1171


Batches:   0%|          | 0/37 [00:00<?, ?it/s]

FAISS index size: 1171


In [5]:
# ========== 5. 質問応答関数 ==========
def ask(question, top_k=3):
    q_emb = embedder.encode([question])
    D, I = index.search(q_emb, top_k)
    context = "\n".join(chunks[i] for i in I[0])

    prompt = f"""以下の小説を読んで、質問に簡潔に短く日本語で答えてください。

{context}

質問: {question}
回答:"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items() if k != "token_type_ids"}

    outputs = model.generate(
        **inputs,
        max_new_tokens=80,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=True,      # サンプル生成
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.5

    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("=====================================")
    print(answer)
    print("=====================================")



In [6]:
# ========== 6. 実行例 ==========
ask("吾輩とは誰かのことか？")
ask("吾輩はどこに向かった？")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


以下の小説を読んで、質問に簡潔に短く日本語で答えてください。

カード

枚折ったそうじゃございませんか」「ええその欠けたところに空也餅くうやもちがくっ付いていましてね」と迷亭はこの質問こそ吾縄張内なわばりうちだと急に浮かれ出す。「色気のない人じゃございませんか、何だって楊子ようじを使わないんでしょう」「今度逢あったら注意しておきましょう」と主人がくすくす笑う。「椎茸で歯がかけるくらいじゃ、よほど歯の性しょうが悪いと思われますが、如何いかがなものでしょう」「善いとは言われますまいな――ねえ迷亭」「善い事はないがちょっと愛嬌あいきょうがあるよ。あれぎり、まだ填つめないところが妙だ。今だに空也餅引掛所ひっかけどころになってるなあ奇観だぜ」「歯を填める小遣こづかいがないの
君の悪口を云う事もあるそうだがね」
「あの娘がか」
「ああ」
「怪けしからん奴だ、悪口を云うなんて。第一それじゃ寒月に意いがないんじゃないか」
「そこがさ、世の中は妙なもので、自分の好いている人の悪口などは殊更ことさら云って見る事もあるからね」
「そんな愚ぐな奴がどこの国にいるものか」と主人は斯様かような人情の機微に立ち入った事を云われても頓とんと感じがない。
「その愚な奴が随分世の中にゃあるから仕方がない。現に金田の妻君もそう解釈しているのさ。戸惑とまどいをした糸瓜へちまのようだなんて、時々寒月さんの悪口を云いますから、よっぽど心の中うちでは思ってるに相違ありませんと」
 主人はこの不可思議

質問: 吾輩とは誰かのことか?
回答: 「いやぁ大変だったのよなぁ~!」と答えて下さい。(笑)
以下の小説を読んで、質問に簡潔に短く日本語で答えてください。

カード

た分量だけツユの嵩かさが増してくる。ところが茶碗の中には元からツユが八分目這入はいっているから、迷亭の箸にかかった蕎麦の四半分しはんぶんも浸つからない先に茶碗はツユで一杯になってしまった。迷亭の箸は茶碗を去さる五寸の上に至ってぴたりと留まったきりしばらく動かない。動かないのも無理はない。少しでも卸おろせばツユが溢こぼれるばかりである。迷亭もここに至って少し※(「足へん+厨」、第3水準1-92-39)躇ちゅうちょの体ていであったが、たちまち脱兎だっとの勢を以て、口を箸の方へ持って行ったなと思う間まもなく、つるつるちゅうと音がして咽喉笛のどぶえが一